In [2]:
import pypots
import numpy as np
import matplotlib.pyplot as plt
import tsdb
import benchpots
from pypots.optim import Adam
from pypots.imputation import CSDI, MRNN, BRITS

from pypots.utils.random import set_random_seed
from pypots.utils.metrics import calc_mae

#from cdrec.python.recovery import centroid_recovery as CDrec

set_random_seed(1234)

2024-10-18 12:51:00 [INFO]: Have set the random seed as 1234 for numpy and pytorch.


In [3]:
ecg200_2 = benchpots.datasets.preprocess_ucr_uea_datasets('ucr_uea_ECG200', 0)
ecg200_2.keys()

2024-10-18 12:51:04 [INFO]: You're using dataset ucr_uea_ECG200, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/ucr_uea_datasets
2024-10-18 12:51:04 [INFO]: Dataset ucr_uea_ECG200 has already been downloaded. Processing directly...
2024-10-18 12:51:04 [INFO]: Dataset ucr_uea_ECG200 has already been cached. Loading from cache directly...
2024-10-18 12:51:04 [INFO]: Loaded successfully!
2024-10-18 12:51:04 [WARNING]: rate is 0, no missing values are artificially added.
2024-10-18 12:51:04 [INFO]: Total sample number: 200
2024-10-18 12:51:04 [INFO]: Training set size: 80 (40.00%)
2024-10-18 12:51:04 [INFO]: Validation set size: 20 (10.00%)
2024-10-18 12:51:04 [INFO]: Test set size: 100 (50.00%)
2024-10-18 12:51:04 [INFO]: Number of steps: 1
2024-10-18 12:51:04 [INFO]: Number of features: 96
2024-10-18 12:51:04 [INFO]: Train set missing rate: 0.00%
2024-10-18 12:51:04 [INFO]: Valid

dict_keys(['n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'val_X', 'val_y', 'test_X', 'test_y', 'label_encoder'])

In [4]:
ecg200 = tsdb.load('ucr_uea_ECG200')

2024-10-18 12:51:31 [INFO]: You're using dataset ucr_uea_ECG200, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/ucr_uea_datasets
2024-10-18 12:51:31 [INFO]: Dataset ucr_uea_ECG200 has already been downloaded. Processing directly...
2024-10-18 12:51:31 [INFO]: Dataset ucr_uea_ECG200 has already been cached. Loading from cache directly...
2024-10-18 12:51:31 [INFO]: Loaded successfully!


In [5]:
X_train = ecg200['X_train'].transpose([0,2,1])
y_train = ecg200['y_train']

X_test = ecg200['X_test'].transpose([0,2,1])
y_test = ecg200['y_test']
n_steps = len(X_test[0])

In [7]:
csdi = CSDI(
    n_steps=n_steps,
    n_features=1,
    n_layers=6,
    n_heads=2,
    n_channels=128,
    d_time_embedding=64,
    d_feature_embedding=32,
    d_diffusion_embedding=128,
    target_strategy="random",
    n_diffusion_steps=50,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=100,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    #patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files 
    saving_path="results/csdi",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2024-10-18 12:51:55 [INFO]: No given device, using default device: cpu
2024-10-18 12:51:55 [INFO]: Model files will be saved to results/csdi/20241018_T125155
2024-10-18 12:51:55 [INFO]: Tensorboard file will be saved to results/csdi/20241018_T125155/tensorboard
/Users/joshua/opt/anaconda3/envs/imputation/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
2024-10-18 12:51:55 [INFO]: CSDI initialized with the given hyperparameters, the number of trainable parameters: 1,693,601


In [9]:
csdi.fit(train_set={'X':X_train})

RuntimeError: Numpy is not available